In [1]:
import modules.encoding as enc
import modules.common as common
import pyodbc
import re
import ast
import encodings
import pandas as pd
import numpy as np
import csv

# this is to have the ability to clear the screen to display the progress bar
from IPython.display import display, clear_output

Loading HotelObservation file

In [2]:
import re
import pandas as pd

BASE_PATH = './'
def remove_white_spaces(text):
    return re.sub(r'\s+', ' ', text)

DATA_COL_NAME = 'combined_text'
LABEL_COL_NAME = 'HotelAccountNo'

top_1000_HotelObservation_file = f'{BASE_PATH}hotel_observation_where_count_greater_than_50_main.csv'
HotelObservations = pd.read_csv(top_1000_HotelObservation_file, delimiter='^', quoting=3, escapechar='\\')

HotelObservations = HotelObservations.applymap(lambda x: remove_white_spaces(str(x)))
HotelObservations.replace('nan', '', inplace=True)


HotelObservations = HotelObservations[[DATA_COL_NAME, LABEL_COL_NAME]]

HotelObservations[DATA_COL_NAME] = \
    HotelObservations[DATA_COL_NAME].str.replace('nan', '', regex=False)
HotelObservations[DATA_COL_NAME] = \
    HotelObservations[DATA_COL_NAME].str.replace('^', '', regex=False)
HotelObservations[DATA_COL_NAME] = \
    HotelObservations[DATA_COL_NAME].str.replace('"', '', regex=False)

HotelObservations = HotelObservations[ HotelObservations[DATA_COL_NAME].apply(lambda x: len( [i for i in x.split(' ') if len(i)>0]) >= 4)]

/tmp/ipykernel_108391/400772228.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  HotelObservations = HotelObservations.applymap(lambda x: remove_white_spaces(str(x)))


is_valid_word: checks if the word is valid. as long as it has digit, letter, or some other chars, it is valid 

extract_values: extract columns from an insertion query. whatever comes after insert into VALUES

try_decoding: try to decode a non-valid word. something like: Ï¿½Ï¿½Ï¿½Ï¿½

try_encoding2: does the same, instead it works good in Hotels dataset.

In [3]:
all_encodings = ['ascii', 'base64_codec', 'big5', 'big5hkscs', 'bz2_codec', 'cp037', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254', 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'cp273', 'cp424', 'cp437', 'cp500', 'cp775', 'cp850', 'cp852', 'cp855', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862', 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp932', 'cp949', 'cp950', 'euc_jis_2004', 'euc_jisx0213', 'euc_jp', 'euc_kr', 'gb18030', 'gb2312', 'gbk', 'hex_codec', 'hp_roman8', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2', 'iso2022_jp_2004', 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'iso8859_2', 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9', 'johab', 'koi8_r', 'kz1048', 'latin_1', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2', 'mac_roman', 'mac_turkish', 'mbcs', 'ptcp154', 'quopri_codec', 'rot_13', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'tactis', 'tis_620', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_32', 'utf_32_be', 'utf_32_le', 'utf_7', 'utf_8', 'uu_codec', 'zlib_codec']

In [4]:
def dict_sort(d, sort_on='keys', desc=False):
    if(sort_on == 'keys'):
        return {k: v for k, v in sorted(d.items(), key=lambda item: item[0], reverse=desc)}
    else:
        return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=desc)}

def get_char_set(text):
    char_set = set()
    for i in text:
        char_set.add(i)
    return char_set

def is_valid_word(word):
    char_set = get_char_set(str(word))
    char_set = [c for c in char_set if c != ' ']
    for s in char_set:
        if(not (s.isdigit() or s.isalpha() or s in ['_', '-', '+', '*', ' ', '·', '.', '(' , ')', "\x04", '//', ',', '&', '@', '/', ';', ':', '=', '\\', '	', ' ', '«',    \
                                                    '|', '!', '$', '#', '?', '%', "'", '"', '`', '’', '´', ' ', '–', '[', ']', '(', ')', '~', '<', '>', '°', '®', '©'   ])):
            return False
    return True

def is_valid_data_row(sentence):
    for word in sentence:
        if(not is_valid_word(word)):
            return False
    return True

def get_valid_word(s):
    word  = ''
    for s in str(s):
        if(s.isdigit() or s.isalpha() or s == ' '):
            word += s
    return word

def extract_values(text):
    values = []
    is_recording = False
    value = ''

    for char in text:
        if char == "'" and not is_recording:  # Start recording
            is_recording = True
            continue
        if char == "'" and is_recording:  # Stop recording
            is_recording = False
            values.append(value)
            value = ''
            continue
        if is_recording:
            value += char

    return values

def try_decoding(s):
    potential_originals = []
    for encoding in all_encodings:
        try:
            potential_original = s.encode(encoding).decode('utf-8')
            if(is_valid_word(potential_original)):
                potential_originals.append((encoding, potential_original))
        except:
            continue
    return potential_originals#[0] if(len(potential_originals) >=1) else ''

def try_decoding2(s):
    potential_originals = {}
    for encoding in all_encodings:
        try:
            potential_original = s.encode('iso8859_1').decode(encoding)
            if(is_valid_word(potential_original)):
                potential_originals[encoding] = potential_original
        except:
            continue
    return potential_originals

def try_decoding3(s):
    potential_originals = []
    for encoding1 in all_encodings:
        for encoding2 in all_encodings:
            try:
                potential_original = s.encode(encoding1).decode(encoding2)
                if(is_valid_word(potential_original)):
                    potential_originals.append((encoding1, encoding2, potential_original))
            except:
                continue
    return potential_originals

get_all_countries = \
[
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Barbuda", "Antigua" "Argentina", "argentina",
    "Armenia", "Australia", "Austria", "™sterreich", "sterrike", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", 
    "Belarus", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", 
    "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Burundi", "Côte d'Ivoire", "Cabo Verde", "cape verde",
    "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China", 
    "Colombia", "Comoros", "Congo", "Congo Brazzaville", "Brazzaville", "Costa Rica", "Croatia", "Cuba", "Cyprus", 
    "Czechia", "Czech Republic", "Democratic Republic of the Congo", "Denmark", "danmark", "denemarken", "džnemark", "faroe islands", "greenland", "Djibouti", "Dominica", 
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", 
    "Eswatini", "Swaziland" "Ethiopia", "ethiopia", "Fiji", "Finland", "France", "Frankreich", "frankrijk", "Gabon", "Gambia", "Georgia", 
    "Germany", "Duitsland", "Deutschland", "geemany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea Bissau", "Bissau" "Guyana", "Haiti", 
    "Holy See", "Honduras", "Hong Kong", "hongkong", "Hungary", "Ungarn", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", 
    "Israel", "Italy", "Ivory", "d'ivoire ", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kuwait", 
    "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", 
    "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", 
    "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", 
    "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Burma" "Namibia", "Nauru", 
    "Nepal", "Netherlands", "nederland", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Korea", "Korea",
    "North Macedonia", "Macedonia", "Norway", "norge" "Oman", "Pakistan", "Palau", "Palestine State", 
    "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "phillippines", "Poland", "Portugal", 
    "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia", 
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", 
    "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", 
    "Solomon Islands", "Somalia", "South Africa", "South Korea", "South Sudan", "Spain", "espaã‘a", "spanien", "spanje", "Sri Lanka", 
    "Sudan", "Suriname", "Sweden", "Sverige", "Switzerland", "schweiz", "zwitserland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", 
    "Timor Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", 
    "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom", "england", "United States of America", "United State",
    "USA", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "ver.k™nigreich", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

def similar_name_places():
    return [
    #city_name, alternative_country, Origin_country
    ("Paris", "USA", "France"),
    ("Jordan", "USA", "Jordan"),
    ("Georgia", "USA", "Georgia"),
    ("Lebanon", "USA", "Lebanon"),
    ("Norway", "USA", "Norway"),
    ("Panama", "USA", "Panama"),
    ("Belgium", "USA", "Belgium"),
    ("Moscow", "USA", "Russia"),
    ("Peru", "USA", "Peru"),
    ("Bolivia", "North Carolina, USA", "Bolivia"),
    ("Jamaica", "New York, USA", "Jamaica"),
    ("Luxembourg", "Wisconsin, USA", "Luxembourg"),
    ("Mexico", "Missouri, USA", "Mexico"),
    ("Sydney", "Montana, USA", "Australia"),
    ("Venezuela", "Colombia", "Venezuela"),
    ("Cuba", "Missouri, USA", "Cuba"),
    ("Barbados", "Ontario, Canada", "Barbados"),
    ("Cambridge", "New Zealand", "United Kingdom"),
    ("Egypt", "Texas, USA", "Egypt"),
    ("Denmark", "South Carolina, USA", "Denmark"),
    ("Wellington", "Tamil Nadu, India", "New Zealand"),
    ("Greece", "New York, USA", "Greece"),
    ("Malta", "Montana, USA", "Malta"),
    ("India", "Missouri, USA", "India"),
    ("Ireland", "Indiana, USA", "Ireland")
]

get_letter_collections = \
    {
        "english": set(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']),
        "finland": set(['Ä', 'Ö']),
        "denmark": set(['Æ', 'Ø', 'Å']),
        "norway": set(['Æ', 'Ø', 'Å']),
        "sweden": set(['Å', 'Ä', 'Ö']),
        "france": set(['À', 'Â', 'Æ', 'Ç', 'É', 'È', 'Ê', 'Ë', 'Î', 'Ï', 'Ô', 'Œ', 'Ù', 'Û', 'Ü', 'Ÿ' ]),        
        "czech republic": set(['Á', 'Č', 'Ď', 'É', 'Ě', 'Í', 'Ň', 'Ó', 'Ř', 'Š', 'Ť', 'Ú', 'Ů', 'Ý', 'Ž']),
        "spain": set(['Ñ']),
        "slovene": set(['Č', 'Š', 'Ž']),
        "croatian": set(['Č', 'Ć', 'Đ', 'Š', 'Ž']),
        "hungarian": set(['Á', 'É', 'Í', 'Ó', 'Ö', 'Ő', 'Ú', 'Ü', 'Ű']),
        "romanian": set(['Ă', 'Â', 'Î', 'Ș', 'Ț', ]), 

        "russia": set(['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я']),
        "japan_1":set(['あ', 'い', 'う', 'え', 'お', 'か', 'き', 'く', 'け', 'こ', 'さ', 'し', 'す', 'せ', 'そ', 'た', 'ち', 'つ', 'て', 'と', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ひ', 'ふ', 'へ', 
            'ほ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん']),
        "japan_2": set(['ア', 'イ', 'ウ', 'エ', 'オ', 'カ', 'キ', 'ク', 'ケ', 'コ', 'サ', 'シ', 'ス', 'セ', 'ソ', 'タ', 'チ', 'ツ', 'テ', 'ト', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'ヒ', 'フ', 'ヘ', 
            'ホ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ヤ', 'ユ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ワ', 'ヲ', 'ン']),\
        "thailand": set(['ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 
            'ม', 'ย', 'ร', 'ฤ', 'ล', 'ฦ', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ']),
    }


alternative_chars = [
        (set(['Ä', 'Æ', 'Å', 'Æ', 'Å', 'Å', 'Ä', 'À', 'Â', 'Æ', 'Á', 'Á', 'Ă', 'Â'  ]), 'A'),
        (set(['Ç', 'Č', 'Č', 'Č', 'Ć' ]), 'C'),
        (set(['Đ', 'Ď' ]), 'D'),
        (set(['É', 'È', 'Ê', 'Ë', 'É', 'Ě', 'É' ]), 'E'),
        (set(['Î', 'Ï', 'Í', 'Î', 'Í' ]), 'I'),
        (set(['Ñ', 'Ň' ]), 'N'),
        (set(['Ö', 'Ø', 'Ø', 'Ö', 'Ô', 'Ó', 'Ó', 'Ö', 'Ő', 'Œ'  ]), 'O'),
        (set(['Ř' ]), 'R'),
        (set(['Š', 'Š', 'Ș', 'Š' ]), 'S'),
        (set(['Ț', 'Ť' ]), 'T'),
        (set(['Ù', 'Û', 'Ü', 'Ú', 'Ů', 'Ú', 'Ü', 'Ű' ]), 'U'),
        (set(['Ÿ', 'Ý' ]), 'Y'),
        (set(['Ž', 'Ž', 'Ž' ]), 'Z')
]
        

def is_chinese(word):
    for character in word:
        if not ('\u4e00' <= character <= '\u9fff'): return False
    return True

def is_japanese(word):
    for character in word:
        if ('\u3040' <= character <= '\u309F' or  # Hiragana
            '\u30A0' <= character <= '\u30FF' or  # Katakana
            '\u4E00' <= character <= '\u9FFF' or  # Common Kanji
            '\u3400' <= character <= '\u4DBF' or  # Extended Kanji A
            '\uF900' <= character <= '\uFAFF' or  # Extended Kanji B
            '\uFF66' <= character <= '\uFF9F'):   # Katakana Phonetic Extensions
            continue
        else: return False
    return True

def is_russian(text):
    for character in text:
        if not ('\u0400' <= character <= '\u04FF' or  # Cyrillic
                '\u0500' <= character <= '\u052F'):  # Cyrillic Supplement
            return False
    return True

def is_thai(text):
    for character in text:
        if not ('\u0E00' <= character <= '\u0E7F'): return False
    return True
    
def is_valid_word_by_chars(word):
    word = word.upper()
    word_length = len(word)    
    english_letters = get_letter_collections['english']
    finland_letters = get_letter_collections['finland']
    denmark_letters = get_letter_collections['denmark']
    norway_letters = get_letter_collections['norway']
    sweden_letters = get_letter_collections['sweden']
    france_letters = get_letter_collections['france']
    czech_letters = get_letter_collections['czech republic']
    spain_letters = get_letter_collections['spain']
    slovene_letters = get_letter_collections['slovene']
    croatian_letters = get_letter_collections['croatian']
    hungarian_letters = get_letter_collections['hungarian']
    romanian_letters = get_letter_collections['romanian']

    valid_chars_counter = len([1 for w in word if w in english_letters]) 
    if(valid_chars_counter == word_length): return (True, 'english')

    if((valid_chars_counter + len([1 for w in word if w in finland_letters])) == word_length): return (True, 'finland')
    if((valid_chars_counter + len([1 for w in word if w in denmark_letters])) == word_length): return (True, 'denmark')
    if((valid_chars_counter + len([1 for w in word if w in norway_letters])) == word_length): return (True, 'norway')
    if((valid_chars_counter + len([1 for w in word if w in sweden_letters])) == word_length): return (True, 'sweden')
    if((valid_chars_counter + len([1 for w in word if w in france_letters])) == word_length): return (True, 'france')
    if((valid_chars_counter + len([1 for w in word if w in czech_letters])) == word_length): return (True, 'czech')
    if((valid_chars_counter + len([1 for w in word if w in spain_letters])) == word_length): return (True, 'spain')
    if((valid_chars_counter + len([1 for w in word if w in slovene_letters])) == word_length): return (True, 'slovene')
    if((valid_chars_counter + len([1 for w in word if w in croatian_letters])) == word_length): return (True, 'croatian')
    if((valid_chars_counter + len([1 for w in word if w in hungarian_letters])) == word_length): return (True, 'hungarian')
    if((valid_chars_counter + len([1 for w in word if w in romanian_letters])) == word_length): return (True, 'romanian')

    if(is_chinese(word)): return (True, 'china')
    if(is_japanese(word)): return (True, 'japan')
    if(is_russian(word)): return (True, 'russia')
    if(is_thai(word)): return (True, 'thailand')

    return (False, '')

def replace_letter_in_word(word,i , l):
    return word[:i] + l + word[i+1:]

def replace_alternative_letter(word):
    for index, l in enumerate(word):
        for letters, alternative in alternative_chars:
            if(l in letters): 
                word = replace_letter_in_word(word, index, alternative)
    return word.lower()

def remove_duplicated_letters(word):
    res_word = ''
    for i in range(len(word)):
        if(i == 0):
            res_word = word[i]; continue
        if(word[i] == word[i-1]): continue
        else: res_word += word[i]
    return res_word

def fine_tune_word(word):
    word = word.upper()
    word = replace_alternative_letter(word)
    word = remove_duplicated_letters(word)
    return word

def list_valid_words(word):
    valid_words = set()
    for _, _, word in try_decoding3(word):
        status, country = is_valid_word_by_chars(word)
        if(status):
            valid_words.add((country, fine_tune_word(word)))
    return valid_words

In [ ]:
{
    "Afghanistan":"",
    "Albania":"",
    "Algeria":"", "Andorra":"", "Angola":"", "Antigua and Barbuda":"", "Barbuda":"", "Antigua":"", "Argentina":"", "argentina":"",
    "Armenia":"", "Australia":"", "Austria":"", "™sterreich":"", "sterrike":"", "Azerbaijan":"", "Bahamas":"", "Bahrain":"", "Bangladesh":"", "Barbados":"", 
    "Belarus":"", "Belgium":"", "Belize":"", "Benin":"", "Bhutan":"", "Bolivia":"", "Bosnia and Herzegovina":"", "Botswana":"", 
    "Brazil":"", "Brunei":"", "Bulgaria":"", "Burkina Faso":"", "Burundi":"", "Côte d'Ivoire":"", "Cabo Verde":"", "cape verde":"",
    "Cambodia":"", "Cameroon":"", "Canada":"", "Central African Republic":"", "Chad":"", "Chile":"", "China":"", 
    "Colombia":"", "Comoros":"", "Congo":"", "Congo Brazzaville":"", "Brazzaville":"", "Costa Rica":"", "Croatia":"", "Cuba":"", "Cyprus":"", 
    "Czechia":"", "Czech Republic":"", "Democratic Republic of the Congo":"", "Denmark":"", "danmark":"", "denemarken":"", "džnemark":"", "faroe islands":"", "greenland":"", "Djibouti":"", "Dominica":"", 
    "Dominican Republic":"", "Ecuador":"", "Egypt":"", "El Salvador":"", "Equatorial Guinea":"", "Eritrea":"", "Estonia":"", 
    "Eswatini":"", "Swaziland":"", "Ethiopia":"", "ethiopia":"", "Fiji":"", "Finland":"", "France":"", "Frankreich":"", "frankrijk":"", "Gabon":"", "Gambia":"", "Georgia":"", 
    "Germany":"", "Duitsland":"", "Deutschland":"", "geemany":"", "Ghana":"", "Greece":"", "Grenada":"", "Guatemala":"", "Guinea":"", "Guinea Bissau":"", "Bissau":"" "Guyana", "Haiti":"", 
    "Holy See":"", "Honduras":"", "Hong Kong":"", "hongkong":"", "Hungary":"", "Ungarn":"", "Iceland":"", "India":"", "Indonesia":"", "Iran":"", "Iraq":"", "Ireland":"", 
    "Israel":"", "Italy":"", "Ivory":"", "d'ivoire":"", "Jamaica":"", "Japan":"", "Jordan":"", "Kazakhstan":"", "Kenya":"", "Kiribati":"", "Kuwait":"", 
    "Kyrgyzstan":"", "Laos":"", "Latvia":"", "Lebanon":"", "Lesotho":"", "Liberia":"", "Libya":"", "Liechtenstein":"", 
    "Lithuania":"", "Luxembourg":"", "Madagascar":"", "Malawi":"", "Malaysia":"", "Maldives":"", "Mali":"", "Malta":"", 
    "Marshall Islands":"", "Mauritania":"", "Mauritius":"", "Mexico":"", "Micronesia":"", "Moldova":"", "Monaco":"", 
    "Mongolia":"", "Montenegro":"", "Morocco":"", "Mozambique":"", "Myanmar":"", "Burma":"", "Namibia":"", "Nauru":"", 
    "Nepal":"", "Netherlands":"", "nederland":"", "New Zealand":"", "Nicaragua":"", "Niger":"", "Nigeria":"", "North Korea":"", "Korea":"",
    "North Macedonia":"", "Macedonia":"", "Norway":"", "norge":"", "Oman":"", "Pakistan":"", "Palau":"", "Palestine State":"", 
    "Panama":"", "Papua New Guinea":"", "Paraguay":"", "Peru":"", "Philippines":"", "phillippines":"", "Poland":"", "Portugal":"", 
    "Qatar":"", "Romania":"", "Russia":"", "Rwanda":"", "Saint Kitts and Nevis":"", "Saint Lucia":"", 
    "Saint Vincent and the Grenadines":"", "Samoa":"", "San Marino":"", "Sao Tome and Principe":"", "Saudi Arabia":"", 
    "Senegal":"", "Serbia":"", "Seychelles":"", "Sierra Leone":"", "Singapore":"", "Slovakia":"", "Slovenia":"", 
    "Solomon Islands":"", "Somalia":"", "South Africa":"", "South Korea":"", "South Sudan":"", "Spain":"", "espaã‘a":"", "spanien":"", "spanje":"", "Sri Lanka":"", 
    "Sudan":"", "Suriname":"", "Sweden":"", "Sverige":"", "Switzerland":"", "schweiz":"", "zwitserland":"", "Syria":"", "Taiwan":"", "Tajikistan":"", "Tanzania":"", "Thailand":"", 
    "Timor Leste":"", "Togo":"", "Tonga":"", "Trinidad and Tobago":"", "Tunisia":"", "Turkey":"", "Turkmenistan":"", 
    "Tuvalu":"", "Uganda":"", "Ukraine":"", "United Arab Emirates":"", "United Kingdom":"", "england":"", "United States of America":"", "United State":"",
    "USA":"", "Uruguay":"", "Uzbekistan":"", "Vanuatu":"", "Venezuela":"", "ver.k™nigreich":"", "Vietnam":"", "Yemen":"", "Zambia":"", "Zimbabwe":""
    }

clean up data set

In [9]:
texts = HotelObservations[DATA_COL_NAME].values
remove_index = []

index = 0
df_length = len(HotelObservations)

for i in range(len(texts)):    
    new_text = []
    for j, word in enumerate(texts[i].split(' ')):
         if (is_valid_word(word)):
            new_text.append(word)
            remove_index.append(j)
    texts[i] = ' '.join(new_text)

    index += 1
    if(index % 10000 == 0):
        clear_output(wait=True)
        display(f'{index} / {df_length}')


'25510000 / 25518392'

count the number of rows with bad words

In [19]:
texts = HotelObservations[DATA_COL_NAME].values

In [20]:
counter = 0
df_length = len(HotelObservations)
index = 0
for text in texts:
    for word in text.split(' '):
         if (not is_valid_word(word)):
             counter += 1
             break
    
    index += 1
    if(index % 10000 == 0):
        clear_output(wait=True)
        display(f'counter:{counter}, {index} / {df_length}')

'counter:0, 25510000 / 25518392'

keeping a list or rows with bad words grup by their countries. This make is easier and faster to find good solution for converting bad words into right unicode.

In [ ]:
rows_with_bad_words_by_country = {}
rows_with_bad_words_by_country['other'] = set()

index = 0
length = len(HotelObservations)
for i, row in HotelObservations.iterrows():
    text = row[DATA_COL_NAME].lower()
    bad_words = [word for word in text.split(' ') if not is_valid_word(word)]
    if (len(bad_words) == 0): continue
    country_in_text = False
    for country in get_all_countries:
        cnty = country.lower()
        if(cnty in text):
            if(rows_with_bad_words_by_country.get(cnty, None) == None): rows_with_bad_words_by_country[cnty] = set()
            for b_word in bad_words:
                rows_with_bad_words_by_country[cnty].add(b_word)
            country_in_text = True
            break
    if(not country_in_text):
        for b_word in bad_words:
            rows_with_bad_words_by_country['other'].add(b_word)
    clear_output(wait=True)
    index+=1
    display(f'iterating {index + 1} / {length}')

In [ ]:
# for word in rows_with_bad_words_by_country['denmark']:
#     if(common.is_all_digit(word)): continue
#     list_valid_words(word)
#     print('-----------')

rows_with_bad_words_by_country['denmark']

In [ ]:
list_valid_words('poissonniã¨re')

analysing rows_with_bad_words_by_country

In [ ]:
count_countries_with_bad_words = {}
for k, v in rows_with_bad_words_by_country.items():
    count_countries_with_bad_words [k] = len(v)

print(len(count_countries_with_bad_words))
# count_countries_with_bad_words

trying to find corrected word for bad_words by language_letter's collection

In [ ]:
def find_correct_word_by_language(word, language_letters):
    for l in set(word):
        if(l.upper() in language_letters): continue
        else: return False
    return True

def find_correct_word(word, language_letters):
    res = set()
    decoded_words = [decoded_word for _, _, decoded_word in try_decoding3(word)]
    for dec_word in decoded_words:
        if(find_correct_word_by_language(dec_word, language_letters)):
            res.add(dec_word)
    return res

index = 0
bad_word_lookup = {}
for country, texts in rows_with_bad_words_by_country.items():
    # if(country.lower() == 'other'): continue
    display(f'country: {country} - {index + 1} / {len(rows_with_bad_words_by_country)} - number of texts in {country}={len(texts)}')
    for text in texts:
        for word in text.split(' '):
            if(is_valid_word(word)): continue
            bad_word_lookup[word] = {}
            for language, language_letters in get_letter_collections.items():
                bad_word_lookup[word][language] = find_correct_word(word, language_letters)

    clear_output(wait=True)
    index += 1

In [ ]:
import pickle

file = open('rows_with_bad_words_by_country.bin', 'w')
pickle.dump(rows_with_bad_words_by_country, file)

file = open('bad_word_lookup.bin', 'w')
pickle.dump(bad_word_lookup, file)


In [ ]:
log = ''
for k, v in bad_word_lookup.items():
    log += f'for bad_word {k} values are: '
    any_value = False
    for language, corrected_word in v.items():
        log += f'language {k}: {", ".join(v)}'
        if (len(v) > 0): any_value=True

    if(any_value): print(log)

In [ ]:
list_valid_words('¬ESK BUD·JOVICE')


# print('----')
# try_decoding3('hiller™d')

In [ ]:
for language, letters in get_letter_collections.items():
    print(language, find_correct_word('chã¢tre', letters))

Looking for rows where they have exceptional country city combination. EX: USA and Paris.

In [ ]:
for i, row in HotelObservations.iterrows():
    text = row[DATA_COL_NAME].lower()
    if (len([word for word in text if not is_valid_word(word)]) == 0): continue
    found = False
    for country in get_all_countries():
        if(country.lower() in text.lower()):
            found = True; break
    if(not found):
        print(text)

In [ ]:
replacements = {
                'Norway': {'Ï¿½':'Ø', '¥':'Ø'},
                'Germany': {'Ã?Â¶':'ö', 'Ã?Â¼':'ü', 'ÃƒÅ¸':'ß', '¥':'ö', '':'ü', 'Ï¿½':'ß', '±':'ß', 'Ã?Â¿Ã?Â':'ß', '³':'ä', '':'ß', '?=':'ü', 'Â':'', '':'ü', '':'ü', '':'', '':''},
                'Wweden': {'Ï¿½':'å', 'Å':'å', '':'å', 'Ã?':'Ö', 'Ã¤':'Ä', 'Ã¶':'Ö'},
                'Finland': {'¦':'P', '':'Å', '„':'ö'},
                'France':{'':'É', },
                'Italy': {'{':'À', '':''},
                'Czech Republic': {'Ã?':'é', 'Ã':'í'}
                }
                
valid_encodings = {
                    'china': ['iso8859_5', 'gbk', 'euc_jis_2004',          'ptcp154', 'shift_jis', 'big5', 'utf_16'],
                    'russia': ['iso8859_5' ],                    
                   }                

In [ ]:
import re

def get_correct_text(text):
    def get_correctly_encoded_word(decoding_result, valid_encodings):
        res = [decoding_result[item] for item in valid_encodings if(decoding_result.get(item, None) != None)]
        return None if len(res) == 0 else res[0]
    word_list = [word for word in text.split(' ')]
    for i, word in enumerate(word_list):
        if(common.is_all_digit(word)):
            continue
        else:
            if(is_valid_word(word)):
                continue
            else:
                decoding_result = try_decoding2(word)

                # if(len(decoding_result) == 0): continue
                if(decoding_result.get('utf_8', None) != None): word_list[i] = decoding_result['utf_8']; continue

                if('china' in word_list and get_correctly_encoded_word(decoding_result, valid_encodings['china']) != None):
                    word_list[i] = get_correctly_encoded_word(decoding_result, valid_encodings['china']); continue

                if('russia' in word_list and get_correctly_encoded_word(decoding_result, valid_encodings['russia']) != None):
                    word_list[i] = get_correctly_encoded_word(decoding_result, valid_encodings['russia']); continue

                if('cp858' in decoding_result.keys()): word_list[i] = decoding_result['cp858']; continue
                if('cp852' in decoding_result.keys()): word_list[i] = decoding_result['cp852']; continue

                new_word = ''
                replacements_countries = replacements.keys()
                for country in replacements_countries:
                    if(country in word_list):
                        print(country, word)
                        new_word = word
                        for k, v in replacements[country].items():
                            new_word = new_word.replace(k, v)
                        break
                if(new_word != ''):
                    word_list[i] = new_word
                    continue

                if('utf_16' in decoding_result.keys()): word_list[i] = decoding_result['utf_16']; continue
                if('iso8859_5' in decoding_result.keys()): word_list[i] = decoding_result['iso8859_5']; continue
                if('iso2022_jp_2' in decoding_result.keys()): word_list[i] = decoding_result['iso2022_jp_2']; continue
    return ' '.join(word_list)

# get_correct_text('INTERCONTINENTAL CHENGDU CENTU 88 SHIJICHENG ROAD, GAOXING DISTRICT, CHENGDU. 88 ZJ, CHENGDU/CHENGDU/Ï¿½É¶Ï¿½ CN China CNY ZJ')
# get_correct_text('NH FURTH NURNBERG KÃ?Â¶NIGSTRASSE 140 NÃ?Â¼RNBERG FÃ?Â¼RTH DE Germany EUR NH')
# get_correct_text('RADISSON BLU HOTEL, SAKARYA HANLIKÃ¶Y MAH. ESKISEHIR CAD 70 SAKARYA TR Turkey EUR ')
# get_correct_text('MOTEL ONE INNSBRUCK 1 SÃ¼DBAHNSTRAÃ?E TOP 1 1 SÃ¼DBAHNSTRAÃ?E TOP 1 INNSBRUCK AT Austria EUR MOT')
get_correct_text('HOTEL WYNDHAM AIRPORT STUTTGART WYNDHAM STUTTGART AIRPORT FLUGHAFENSTRAÃƒÅ¸E 51 70629 STUTTGART DE Germany EUR')

In [ ]:
# refine hotels data

item_count = {}

for index, row in HotelObservation.iterrows():    
    for col in row.values:
        col = str(col)
        if(common.is_all_digit(col)):
            pass
        else:
            if(is_valid_word(col)):
                pass
            else:
                country = str(row['countryname']).lower().strip()
                decoding_result = try_decoding2(col)

                if(len(decoding_result) == 0): continue                
                if(decoding_result.get('utf_8', None) != None or decoding_result.get('utf_16', None) != None): continue

                if(country in ['china', 'russia'] ): continue #['norway', 'china', 'russia', 'germany', 'sweden']
                if('cp858' in decoding_result.keys()): continue
                if('cp852' in decoding_result.keys()): continue

                if(country in replacements.keys()):
                    for k, v in replacements[country].items():
                        col = col.replace(k, v)
                if(is_valid_word(col)): continue

                if('iso8859_5' in decoding_result.keys()): continue

                if('iso2022_jp_2' in decoding_result.keys()): continue                

                # get_valid_word(col)

                print(country, '-----', col, '\n', decoding_result)
                print()
                # is_valid = [1 for val_enc in valid_encodings[country] if(val_enc in decoding_result.keys())]
                # if(sum(is_valid) > 0): continue

                for val_enc in decoding_result.keys():
                    if(val_enc not in item_count.keys()):
                        item_count[val_enc] = 1
                    else:
                        item_count[val_enc] += 1

In [ ]:
dict_sort(item_count, sort_on='values', desc=True)

In [ ]:
corrupted_string = "LULEÏ¿½"

# Assume it was meant to be UTF-8 but was misinterpreted
# First, we need to encode it back to bytes assuming it was misinterpreted as Windows-1252 or a similar encoding
bytes_string = corrupted_string.encode('windows-1252')

# Now, we decode it as UTF-8
correct_string = bytes_string.decode('utf-8')

print(correct_string)

In [ ]:
xxx = "[GUEST HOUSE] ÇÑ¸¶À½ "

print(is_valid_word(xxx), '\n')

print(try_decoding2(xxx))

In [ ]:
# ['utf_8', 'utf_16', 'mac_latin2'] 'shift_jis'
# {'China', 'shift_jis}

In [ ]:
xxx.encode('latin1').decode('utf-8')

In [ ]:
for c in '''RH\NPERLE BQD NEUSTADT''':
    print( c)

In [ ]:
repr(str('RH\NPERLE BQD NEUSTADT'))

whenever there is an invalid word, we use try_decoding to get the right alternatives for that word. and then we make new insertion with the alternative/alternatives words and return the list.

dump_into_sql: runs the query

In [ ]:
def get_new_insertions(row):
    columns = extract_values(row.split(' VALUES ')[1])
    new_insertions = []
    for col in columns:
        if(not(is_valid_word(col))):
            decoded_values = try_decoding(col)
            if(len(decoded_values) > 0):
                for decoded_value in decoded_values:
                    new_insert = row
                    new_insert = new_insert.replace(col, decoded_value)
                    new_insertions.append(new_insert)
    return new_insertions

def dump_into_sql(insertions):
    for insertion in insertions:
        cursor.execute(insertion)
        connection.commit()

list of available tables in world_db

In [ ]:
table_names = ['cityNames'] # ['locales', 'regions', 'regionNames', 'countries', 'countryNames', 'cities', 

In [ ]:
for table in table_names:
    rows = get_table_rows(table)
    print(table, len(rows))
    i = 0
    # try:
    for row in rows:
        insertions = [row]
        new_insertions = get_new_insertions(row)
        if(len(new_insertions) > 0):
            for ins in new_insertions:
                insertions.append(ins)

        i += 1
        if(i % 100 == 0):
            clear_output(wait=True)
            display(f'Current row: {i}')

        dump_into_sql(insertions)
    # except Exception as e:
        # print('error', e)
        # print(insertions)
        # print(new_insertions)

In [ ]:
from gensim.models import Word2Vec

# Sample sentences
sentences = [
    ["apple", "fruit", "tasty"],
    ["banana", "fruit", "yellow"],
    ["salmon", "fish", "pink"],
    ["trout", "fish", "freshwater"]
]

# Train a Word2Vec model
model = Word2Vec(sentences, vector_size=15, window=2, min_count=1, workers=4)
# model.save("word2vec_example.model")

# Find vector for a word
vector_apple = model.wv['apple']

# Find similar words
similar_to_apple = model.wv.most_similar('apple', topn=3)

print(similar_to_apple)
